In [21]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [58]:
app = dash.Dash(__name__)
year_list = [i for i in range(1980, 2024, 1)]
app.config.suppress_callback_exceptions = True
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
data = pd.read_csv(URL)
data.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [140]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1(' Automobile Sales Statistics Dashboard', 
                                    style={'textAlign': 'center', 'color': '#503D36',
                                    'font-size': 24}),
                                dcc.Dropdown(id='dropdown-statistics', 
                                    options=[
                                        {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                                        {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                                    ],
                                    placeholder='Select a report type',
                                    value="Select Statistics",
                                    style={'width':"80%",'padding':'3px','font-size':"20px",'text-align-last':'center'}),
                                dcc.Dropdown(id='select-year', 
                                    options=[{'label': i, 'value': i} for i in year_list],
                                    placeholder='Select-year',
                                    value = 'Select-year',
                                    style={'width':"80%",'padding':'3px','font-size':"20px",'text-align-last':'center'}),
                                html.Div([
                                    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
    ])
])

In [141]:
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))

def update_input_container(selected_stats):
    if selected_stats =='Yearly Statistics': 
        return False
    else: 
        return True

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
    Input(component_id='select-year', component_property='value')])

def update_output_container(selected_statistics, select_year):
    
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        #URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
        #data = pd.read_csv(URL)
        recession_data = data[data['Recession'] == 1]
        #Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
        # grouping data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile Sales across the Years"))
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                 
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Vehicle Type and Sales")
                )
        exp_rec= recession_data.groupby('Vehicle_Type')["Advertising_Expenditure"].sum().reset_index()  
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title="Expenditure of Each Type during Recesion")
                )
        unemp_data= recession_data.groupby(['Vehicle_Type', 'unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x='Vehicle_Type',
        y='Automobile_Sales',
        color='unemployment_rate',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))
        

        return[
                html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})
        ]

    elif (select_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == select_year]
                                
    
    # Plot 1 :Yearly Automobile sales using line chart for the whole period.
    
        # grouping data for plotting.
        # Hint:Use the columns Year and Automobile_Sales.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas,x='Year',y='Automobile_Sales',title='Yearly Sales'))
  
    # Plot 2 :Total Monthly Automobile sales using line chart.
 
            # grouping data for plotting.
        # Hint:Use the columns Month and Automobile_Sales.
        mas=data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
            x='Month',
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))
    
    # Plot bar chart for average number of vehicles sold during the given year

            # grouping data for plotting.
            # Hint:Use the columns Year and Automobile_Sales
        avr_vdata=yearly_data.groupby('Vehicle_Type')[]
        Y_chart3 = dcc.Graph( figure=.................,title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

    # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
            # grouping data for plotting.
            # Hint:Use the columns Vehicle_Type and Advertising_Expenditure
            exp_data=yearly_data.groupby('.......')
        Y_chart4 = dcc.Graph(
            figure=px.pie(........., 
            values='......',
            names='......',
            title='Total Advertisment Expenditure for Each Vehicle'))
    return [
                html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display':'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                ]
        


In [142]:
if __name__ == '__main__':
    app.run_server(jupyter_mode='external')